In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

import base64
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "snhu2022"
shelter = AnimalShelter(username, password)

# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read_all({}))

animalList = ["Water Rescue","Mountain or Wilderness Rescue",
              "Disaster Rescue or Individual Tracking", "Reset"] # Used in radio items

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

# Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('CS-340 Project Two Reames'))),
    html.Hr(),
    html.Center([html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
             style={"width":"10%"})]), # width set to reduce image size
    # Radio item default set to "All" so table displays as unfiltered initially
    html.Div([dcc.RadioItems(options=[{'label':i, 'value':i} for i in animalList], 
                           value="All", id="animal-buttons")]),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
# Interactive features of table
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        row_deletable=False,
        row_selectable="single",
        column_selectable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10,
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################

# Filtering options
@app.callback(
    Output('datatable-id',"data"),
    [Input('animal-buttons','value')]
)
def filter_dashboard(value):
    # Updates data table based on dropdown selection
    if ((value == "All") or (value == "Reset")):
        df = pd.DataFrame(shelter.read_all({}))
        
    elif (value == "Water Rescue"):
        df = pd.DataFrame(list(shelter.read_all(
            {"animal_type":"Dog",
             "breed":{"$in":["Labrador Retriever Mix","Chesapeake Bay Retriever","Newfoundland"]},
             "sex_upon_outcome":"Intact Female",
             "age_upon_outcome_in_weeks":{"$gte":26},
             "age_upon_outcome_in_weeks":{"$lte":156}})))
        
    elif (value == "Mountain or Wilderness Rescue"):
        df = pd.DataFrame(list(shelter.read_all(
            {"animal_type":"Dog",
             "breed":{"$in":["German Shepherd","Alaskan Malamute","Old English Sheepdog",
                             "Siberian Husky","Rottweiler"]},
             "sex_upon_outcome":"Intact Male",
             "age_upon_outcome_in_weeks":{"$gte":26},
             "age_upon_outcome_in_weeks":{"$lte":156}})))
        
    elif (value == "Disaster Rescue or Individual Tracking"):
        df = pd.DataFrame(list(shelter.read_all(
            {"animal_type":"Dog",
             "breed":{"$in":["Doberman Pinscher","German Shepherd","Golden Retriever",
                             "Bloodhound","Rottweiler"]},
             "sex_upon_outcome":"Intact Male",
             "age_upon_outcome_in_weeks":{"$gte":20},
             "age_upon_outcome_in_weeks":{"$lte":300}})))
        
    return df.to_dict('records')

# To reset page to first page whenever a filter is applied
@app.callback(
    Output('datatable-id','page_current'),
    [Input('animal-buttons','value')]
)
def reset_page(filter_dashboard):
    return 0

# Highlights selected cell
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Displays pie chart with different breeds by % per page
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)
 
    return [
       dcc.Graph(            
        figure = px.pie(dff, names='breed')
       )    
    ]

# Displays map with markers for all animals on current page
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    # Markers include position, name of animal (if not blank), and breed
    markers = [dl.Marker(position=[dff.iloc[i,13], dff.iloc[i,14]],
                        children=[dl.Popup([html.H1(dff.iloc[i,9]),
                                            html.P(dff.iloc[i,4])])]
                        ) for i, row in dff.iterrows()]
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[dff.iloc[0,13],dff.iloc[0,14]], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.LayerGroup(markers)
        ])
    ]


app